In [11]:
import os
import numpy as np
from urllib.parse import urlencode 
import pandas as pd 
from astroquery.cadc import Cadc
from astropy.table import Table as A_Table
from IPython.display import Image, display, HTML, clear_output
from IPython.core.display import HTML
import astropy.io.votable.tree 
from astropy.io.votable.tree import VOTableFile, Resource, Table, Field
from astropy import units as u 
from astropy.coordinates import SkyCoord
from astropy.time import Time, TimeDelta
from cadcutils import net
import cadctap
from cadctap.core import main_app
from cadctap import CadcTapClient
from six import  BytesIO
import glue_jupyter as gj
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, interactive_output
import csv
import base64


   

params = {'claimedtype' : 'Ia', 'format' : 'csv' }
Catalog_domain ='https://api.sne.space/catalog'
query_str  = urlencode(params)
url = '{}?{}&first'.format(Catalog_domain,query_str)
data = pd.read_csv(url)

##set the widgets here, they will be used later as input 
min_redshift = widgets.FloatSlider(description='min_redshift',min=0.0, max=1.9, step=0.1, value = 0.2, continuous_update=False)
min_redshift.style.handle_color = 'lightblue'
max_redshift = widgets.FloatSlider(description='max_redshift',min=0.0, max=2.0, step=0.1, value = 0.3, continuous_update=False)
max_redshift.style.handle_color = 'lightblue'
event = widgets.Dropdown(options=['SNLS','DES'], description='Event')
days_before_max_date= widgets.IntSlider(description='days_before_max_date',min=0, max=200, step=1, value = 20, continuous_update=False)
days_before_max_date.style.handle_color = 'moccasin'
days_after_max_date= widgets.IntSlider(description='days_after_max_date',min=0, max=200, step=1, value = 100, continuous_update=False)
days_after_max_date.style.handle_color = 'moccasin'
ui_catalog = widgets.HBox([min_redshift, max_redshift, event])
ui_db = widgets.HBox([days_before_max_date, days_after_max_date ])

##
# Creating a votable file 
# @param: fields_dictionary , df 
# @return: no return type
def create_a_new_votable(field_dictionary,df):
    print("Creating the votable file for the tmp table.")
    num_row = df.shape[0]  #get total number of rows in the dataframe  
    votable = VOTableFile()
    resource = Resource()
    votable.resources.append(resource)
    table = Table(votable)
    resource.tables.append(table)
    list =[]
    for key, value in field_dictionary.items():
        if(len(value)==2):
            list.append(Field(votable, name = key, datatype= value[0], arraysize = value[1]))
        elif(len(value)==3):
            list.append(Field(votable, name = key, datatype= value[0], arraysize = value[1], xtype = value[2], config={'version_1_2_or_later':True}))#hacky way 
        else:
            list.append(Field(votable, name = key, datatype= value))
                
    table.fields.extend(list)
    table.create_arrays(num_row)
    ### replace the NaN value in satrt_date and end_date with 0.0
    
    for i in range(0,num_row):
        sn = sn_data.iloc[i]
        table.array[i] = (str(sn["event"]),str(sn["alias"]), str(sn["catalog"]), str(sn["claimedtype"]), sn["dec"],
                          str(sn["discoverdate"]),str(sn["discoverer"]),str(sn["download"]),
                          sn["ebv"],str(sn["host"]),str(sn["hostdec"]),sn["hostoffsetang"], sn["hostoffsetdist"],
                          str(sn["hostra"]),str(sn["instruments"]), sn["lumdist"], sn["maxabsmag"], sn["maxappmag"],
                          str(sn["maxdate"]), str(sn["name"]),str(sn["photolink"]),
                          sn["ra" ],sn["radiolink"],  sn["redshift"],str(sn["references"]),sn["sources"], 
                          str(sn["spectralink"]),sn["velocity"],sn["xraylink"],sn["start_date"],sn["end_date"],
                          [sn["start_date"],sn["end_date"]])
    votable.to_xml("tmp/new_votable.xml")


### button waits on a onclick event    
button = widgets.Button(description='Load Subsets',
                        layout= widgets.Layout(display='flex',
                                               flex_flow='column',
                                               align_items='stretch',
                                               width='50%'),
                        style=widgets.ButtonStyle(button_color='lightblue'))
out = widgets.Output()
###


##
# Interactive function, hinde and display different type of graphs 
# @param: Scatter(widget), Histogram(widget)
# @return: no return type 
def show_hide_graphs(Graph_Type):
    print("")
    if (Graph_Type == 'Scatter'):
        scatter_viewer = app.scatter2d(x = 'Int. Time',y = 'Start Date', data = out_data, show = True )
        histogram_viewer = app.histogram1d(x = 'Int. Time', data = out_data, show = False )
    else:
        scatter_viewer = app.scatter2d(x = 'Int. Time',y = 'Start Date', data = out_data, show = False )
        histogram_viewer = app.histogram1d(x = 'Int. Time', data = out_data, show = True )    



##
# Interactive function, used to process data, and call other functions, also creating 5 widgets 
# @param: min_redshift(widget), max_redshift, event, days_before_max_date, days_after_max_date
# @return: no return type 
def processing_data(min_redshift, max_redshift, event,days_before_max_date, days_after_max_date):
    global out_data, sn_data, click_flag, app
    click_flag = 0
    button.click()  # triger the update subsets button to reaload 
    print("set constraints on the supernova catalog table")
    sn_data = data.copy()
    button.click() 
    sn_data = sn_data[(sn_data['redshift']>min_redshift) & (sn_data['redshift']<max_redshift)] 
    sn_data = sn_data[sn_data['event'].str.startswith(event)]
    sn_data = sn_data[sn_data['discoverdate']< sn_data['maxdate']]
    sn_data["ra"] =SkyCoord(sn_data["ra"], sn_data["dec"],unit=(u.hourangle,u.deg)).ra.degree
    sn_data["dec"] = SkyCoord(sn_data["ra"], sn_data["dec"],unit=(u.hourangle,u.deg)).dec.degree
    sn_data["maxdate"] = sn_data["maxdate"].str.replace('/','-')
    print('Number of supernova: {}'.format(len(sn_data)))
    print("Calculating coords and start end date")
    for index, row in sn_data.iterrows():
        if(str(row["maxdate"])!='nan'):
            processed_maxdate = Time(row["maxdate"], format='isot')
            sn_data.at[index, 'start_date'] =  (processed_maxdate - TimeDelta(days_before_max_date,format='jd')).mjd  
            sn_data.at[index, 'end_date'] = (processed_maxdate + TimeDelta(days_after_max_date, format='jd')).mjd
        else: ##set place holder 
            sn_data.at[index, 'start_date'] =  0.0  
            sn_data.at[index, 'end_date'] = 0.1       
    button.click()  
    sn_data.dropna(subset=['maxdate']) 
        
    field_dictionary = {
        "event":["char", "*"],
        "alias":["char", "*"],
        "catalog":["char", "*"],
        "claimedtype":["char", "*"],
        "dec":"double",
        "discoverdate":["char", "*"],
        "discoverer":["char", "*"],
        "download":["char","*"],
        "ebv":"double" ,        
        "host":["char","*"],  
        "hostdec":["char","*"],  
        "hostoffsetang":"double",  
        "hostoffsetdist":"double", 
        "hostra":["char","*"],       
        "instruments":["char","*"],   
        "lumdist":"double",           
        "maxabsmag":"double",          
        "maxappmag":"double",
        "maxdate" :["char","*"], 
        "name" :["char","*"],
        "photolink":["char","*"],    
        "ra" :"double",
        "radiolink":"double",  
        "redshift":"double",          
        "reference":["char","*"],      
        "sources":"double",              
        "spectralink":["char","*"],      
        "velocity":"double",                
        "xraylink":"double",                
        "start_date":"double",
        "end_date": "double",
        "start_end_date":["double", "2", "interval"]   
    }

    ### create a votable file
    create_a_new_votable(field_dictionary, sn_data)   
    
    anonSubject = net.Subject()
    client = CadcTapClient(anonSubject, resource_id ='ivo://cadc.nrc.ca/tap')
    
    query ="""SELECT Observation.observationURI AS "Preview",
    Observation.collection AS "Collection",
    Observation.observationID AS "Obs. ID" ,
    Plane.productID AS "Product ID",
    COORD1(CENTROID(Plane.position_bounds)) AS "RA (J2000.0)",
    COORD2(CENTROID(Plane.position_bounds)) AS "Dec. (J2000.0)",
    Plane.time_bounds_lower AS "Start Date",
    Observation.instrument_name AS "Instrument",
    Plane.time_exposure AS "Int. Time",
    Observation.target_name AS "Target Name",
    Plane.energy_bandpassName AS "Filter",
    Plane.calibrationLevel AS "Cal. Lev.",
    Observation.type AS "Obs. Type",
    Plane.energy_bounds_lower AS "Min. Wavelength",
    Plane.energy_bounds_upper AS "Max. Wavelength",
    Observation.proposal_id AS "Proposal ID",
    Observation.proposal_pi AS "P.I. Name",
    Plane.dataRelease AS "Data Release",
    AREA(Plane.position_bounds) AS "Field of View",
    Plane.position_bounds AS "Polygon",
    Plane.position_sampleSize AS "Pixel Scale",
    Plane.energy_resolvingPower AS "Resolving Power",
    Plane.time_bounds_upper AS "End Date",
    Plane.dataProductType AS "Data Type",
    Observation.target_moving AS "Moving Target",
    Plane.provenance_name AS "Provenance Name",
    Observation.intent AS "Intent",
    Observation.target_type AS "Target Type",
    Observation.target_standard AS "Target Standard",
    Observation.target_keywords AS "Target Keywords",
    Observation.sequenceNumber AS "Sequence Number",
    Observation.algorithm_name AS "Algorithm Name",
    Observation.proposal_title AS "Proposal Title",
    Observation.proposal_keywords AS "Proposal Keywords",
    Plane.position_resolution AS "IQ",
    Observation.instrument_keywords AS "Instrument Keywords",
    Observation.environment_tau AS "Tau",
    Plane.energy_transition_species AS "Molecule",
    Plane.energy_transition_transition AS "Transition",
    Observation.proposal_project AS "Proposal Project",
    Plane.energy_emBand AS "Band",
    Plane.provenance_version AS "Prov. Version",
    Plane.provenance_project AS "Prov. Project",
    Plane.provenance_runID AS "Prov. Run ID",
    Plane.provenance_lastExecuted AS "Prov. Last Executed",
    Plane.energy_restwav AS "Rest-frame Energy",
    Observation.requirements_flag AS "Quality",
    Plane.publisherID AS "Publisher ID" FROM caom2.Plane AS Plane
    JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID join tap_upload.tmptable as Tmp on
    (INTERSECTS(Tmp.start_end_date, Plane.time_bounds_samples) = 1
    AND CONTAINS( POINT('ICRS',Tmp.ra, Tmp.dec ), Plane.position_bounds ) = 1)
    WHERE (LOWER(Plane.energy_bandpassName) LIKE 'r%'
    AND collection = 'CFHT'
    AND calibrationLevel >= 2
    AND (Plane.quality_flag IS NULL OR Plane.quality_flag != 'junk'))
    ORDER BY time_bounds_lower ASC"""

    def_table = os.path.join('tmp','new_votable.xml')
    print("fetching data from the database...")
    client.query(query, output_file= "tmp/output_file.csv", response_format= 'csv', tmptable='tmptable:' + def_table, timeout = 30)

    
    ### Hack: remove the triple quotes in the output csv file
    results = pd.read_csv('tmp/output_file.csv')
    count_row = len(results.index)
    results.columns = [col.replace('"', '') for col in results.columns]
    results.to_csv('tmp/output_file.csv',index=False)
    print('Total row count: {}'.format(str(count_row)))
    
      
    ### display data using glue
    app = gj.jglue()
    out_data = app.load_data('tmp/output_file.csv')
    print("displaying data.")
    button.click()
    click_flag = 1
    interact(show_hide_graphs, Graph_Type = ['Scatter','Histogram'] )

    
### this interactive_output function allowas me to modify the layout of the param widgets    
output = widgets.interactive_output(processing_data, {'min_redshift': min_redshift,
                                                    'max_redshift': max_redshift,
                                                    'event': event,
                                                    'days_before_max_date': days_before_max_date ,
                                                    'days_after_max_date': days_after_max_date})
### unlike interact(),  interactive_output() does not display the widgets itself
display(ui_catalog,ui_db,output)    
    

    
##
# allow user to select a way to view the images using the provided urls
# @param: Obtain_Images(list widgest)
# @return: no return type   
def select_method(Obtain_Images):
    #download all the urls as a text file 
    if Obtain_Images == 'Download Images':
        txt = ""
        for key in url_dictionary:
            txt = txt + str(key) + '\n'
        b64 = base64.b64encode(txt.encode())
        payload = b64.decode()
        html = '<a download="testfile" href="data:text;base64,{payload}" target="_blank">Download Subset</a>'
        html = html.format(payload=payload)
        display(HTML(html))
    #display the actual images when the number of images is smaller than 10.    
    else:
        if len(url_dictionary) > 10:
            print("Can not display more than 10 images")
        else:
            for value in url_dictionary.values():
                print(str(value[1]))
                link = '<a href="{}">{}</a>'.format(str(value[0]), str(value[0]))
                display(HTML(link))
                display(Image(url = str(value[0]) ,width=100, height=100,unconfined=True))

    
##
# interactive function, used to display and create a dataframe of the selected data 
# @param: subset(list widgest)
# @return: no return type   
def preview_subset(subset):
    global url_dictionary
    lable = subset.split(' ')
    subset_data = {'Target Name':out_data.subsets[int(lable[1])-1]['Target Name'],
                   'publisherID':out_data.subsets[int(lable[1])-1]['Publisher ID'],
                   'Min. Wavelength':out_data.subsets[int(lable[1])-1]['Min. Wavelength'], 
                   'Start Date':out_data.subsets[int(lable[1])-1]['Start Date'],
                   'Int. Time':out_data.subsets[int(lable[1])-1]['Int. Time'],
                   'Product ID':out_data.subsets[int(lable[1])-1]['Product ID'],
                   'Filter':out_data.subsets[int(lable[1])-1]['Filter']} 
    subset_df = pd.DataFrame(subset_data) 
    cadc = Cadc()
    subset_table = A_Table.from_pandas(subset_df)
    url_dictionary = {}
    for idx in range(len(subset_table)):
        # The get_data_urls only accepts a table as input
        url = cadc.get_data_urls(subset_table[idx:idx + 1], include_auxiliaries=True)
        fz_url = next((u for u in url if '.fz' in u),None)
        jpg_url = next((u for u in url if '1024.jpg' in u),None)
        #build the dictionary 
        if jpg_url:
            url_dictionary[fz_url] = [jpg_url,subset_df['Target Name'][idx]]          
    display(subset_df)
    interact(select_method, Obtain_Images = ['Download Images','Display Images'])

    
##
# function will get triggered by a on click event 
# @param: no param 
# @return: no return type 
def on_button_clicked(_):
    # "linking function with output"
    with out:
        # what happens when we press the button
        clear_output()
        if(len(out_data.subsets)!= 0 and click_flag == 1):
            list =[]
            for i in range(len(out_data.subsets)): 
                list.append(out_data.subsets[i].label) 
            interact(preview_subset,subset=list)
        elif(len(out_data.subsets) == 0 and click_flag == 1) :
            print('No subsets created')
        

# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])    


    
#note : data.to_dataframe() convets a glue.Data object to pandas frame     

Output()